In [24]:
from spacerocks.nbody import Simulation, Integrator, Force
from spacerocks.time import Time
from spacerocks.spice import SpiceKernel
from spacerocks import SpaceRock



In [25]:
kernel = SpiceKernel(config="/Users/thomasruch/srnew1/spacerocks/config.toml")

Loading configuration from /Users/thomasruch/srnew1/spacerocks/config.toml

Configuration loaded:
  Kernel paths: ["/Users/thomasruch/Gerdes", "/Users/thomasruch/.spacerocks/spice/"]
  Download directory: "/Users/thomasruch/.spacerocks/spice"
  Auto-download: true

Processing kernel: latest_leapseconds.tls
✓ Found existing kernel at: /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls
Loading kernel: /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls

Processing kernel: de440s.bsp
✓ Found existing kernel at: /Users/thomasruch/Gerdes/de440s.bsp
Loading kernel: /Users/thomasruch/Gerdes/de440s.bsp

Processing kernel: earth_1962_240827_2124_combined.bpc
✓ Found existing kernel at: /Users/thomasruch/Gerdes/earth_1962_240827_2124_combined.bpc
Loading kernel: /Users/thomasruch/Gerdes/earth_1962_240827_2124_combined.bpc


In [17]:
kernel

SpiceKernel:
  - /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls
  - /Users/thomasruch/Gerdes/de440s.bsp
  - /Users/thomasruch/Gerdes/earth_1962_240827_2124_combined.bpc

In [18]:
# Create a simulation with the giant planets
epoch = Time.now()
sim = Simulation.giants(epoch=epoch, reference_plane="ECLIPJ2000", origin="SSB")

# Add relativistic effects
sim.add_force(Force.solar_gr())
sim.add_force(Force.solar_j2())

# Configure integration
sim.set_integrator(Integrator.ias15(timestep=1.0))  # 1-day timestep
sim.move_to_center_of_mass()

# Integrate forward 1 year
future_epoch = epoch + 365.25
sim.integrate(future_epoch)

# Get final positions
jupiter = sim.get_particle("jupiter barycenter")
saturn = sim.get_particle("saturn barycenter")
print(f"Jupiter position: {jupiter.position}")
print(f"Saturn position: {saturn.position}")

Jupiter position: (-1.8481886625435187, 4.876897193602802, 0.021132464797636318)
Saturn position: (9.493932925459712, 0.36919218835859585, -0.38442713519963534)


In [22]:
# Start with empty simulation
sim = Simulation()
epoch = Time.now()

print(epoch)

# Add Sun and a custom body
sun = SpaceRock.from_spice("sun", epoch=epoch, reference_plane="ECLIPJ2000", origin="SSB")
sim.add(sun)

# Add custom body on eccentric orbit
body = SpaceRock.from_kepler(
    name="custom_body",
    q=2.5,       # perihelion distance [AU]
    e=0.85,      # eccentricity
    inc=0.4,     # inclination [rad]
    node=1.2,    # longitude of ascending node [rad]
    arg=2.1,     # argument of perihelion [rad]
    true_anomaly=0.0,  # true anomaly [rad]
    epoch=epoch,
    reference_plane="ECLIPJ2000",
    origin="SSB"
)
body.set_mass(1e-10)  # Set mass in solar masses
sim.add(body)

# Move to center of mass and integrate
sim.move_to_center_of_mass()
sim.integrate(epoch + 100.0)  # Integrate for 100 days

Time: 2460697.2771643517 UTC JD


In [23]:
sim.get_particle(body.name)

SpaceRock: custom_body
position: [[-1.9230760017109423, -1.8506746966685648, -0.27745451535599974]]
velocity: [[0.0055958804609810954, -0.009755876174290613, -0.008712020275687881]]
epoch: 2460797.2771643517 UTC JD
reference_plane: J2000
origin: simulation_barycenter

In [38]:
from spacerocks import RockCollection

In [39]:
epoch = Time.now()
collection = RockCollection()


In [40]:
Jupiter = SpaceRock.from_horizons("Jupiter Barycenter", epoch)
Neptune = SpaceRock.from_horizons("Neptune Barycenter", epoch)
Mars = SpaceRock.from_horizons("Mars Barycenter", epoch)

In [41]:
collection.add(Jupiter)
collection.add(Neptune)
collection.add(Mars)

In [42]:
collection.a()

array([ 5.17713205, 30.06945221,  1.51794713])

In [44]:
under_10_collection = collection.filter(collection.a() < 10)

In [45]:
under_10_collection

RockCollection: 2 rocks

In [47]:
for rock in under_10_collection:
    print(rock.a())

5.177132054094702
1.517947133743725


In [51]:
mask = [rock.e() > 0.05 for rock in collection]
filtered_collection = collection.filter(mask)

In [54]:
filtered_collection[0].e()

0.09827502028901582

In [1]:
from spacerocks.spice import SpiceKernel
from spacerocks import SpaceRock
from spacerocks.time import Time

In [2]:
# Manually instantiate a SpiceKernel object, and load relevant spice kernels located on your local machine 
kernel = SpiceKernel()
kernel.load("/Users/thomasruch/Gerdes/leap_seconds.tls")
kernel.load("/Users/thomasruch/Gerdes/de440s.bsp")

Loading kernel: /Users/thomasruch/Gerdes/leap_seconds.tls
Loading kernel: /Users/thomasruch/Gerdes/de440s.bsp


In [3]:
kernel

SpiceKernel:
  - /Users/thomasruch/Gerdes/leap_seconds.tls
  - /Users/thomasruch/Gerdes/de440s.bsp

In [4]:
kernel1 = SpiceKernel.from_config("/Users/thomasruch/srnew1/spacerocks/config.toml")

Loading configuration from /Users/thomasruch/srnew1/spacerocks/config.toml

Configuration loaded:
  Kernel paths: ["/Users/thomasruch/Gerdes", "/Users/thomasruch/.spacerocks/spice/"]
  Download directory: "/Users/thomasruch/.spacerocks/spice"
  Auto-download: true

Processing kernel: latest_leapseconds.tls
➜ Downloading kernel...
    Saving to /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls
    Download complete
Loading kernel: /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls

Processing kernel: de440s.bsp
✓ Found existing kernel at: /Users/thomasruch/Gerdes/de440s.bsp
Loading kernel: /Users/thomasruch/Gerdes/de440s.bsp

Processing kernel: earth_1962_240827_2124_combined.bpc
✓ Found existing kernel at: /Users/thomasruch/Gerdes/earth_1962_240827_2124_combined.bpc
Loading kernel: /Users/thomasruch/Gerdes/earth_1962_240827_2124_combined.bpc


In [5]:
kernel1 

SpiceKernel:
  - /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls
  - /Users/thomasruch/Gerdes/de440s.bsp
  - /Users/thomasruch/Gerdes/earth_1962_240827_2124_combined.bpc

In [6]:
kernel2 = SpiceKernel.defaults(download=True)


Using default configuration:
  Kernel paths: ["/Users/thomasruch/.spacerocks/spice"]
  Download directory: "/Users/thomasruch/.spacerocks/spice"
  Auto-download: true

Processing kernel: latest_leapseconds.tls
✓ Found existing kernel at: /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls
Loading kernel: /Users/thomasruch/.spacerocks/spice/latest_leapseconds.tls

Processing kernel: de440s.bsp
➜ Downloading kernel...
    Saving to /Users/thomasruch/.spacerocks/spice/de440s.bsp
    Download complete
Loading kernel: /Users/thomasruch/.spacerocks/spice/de440s.bsp

Processing kernel: earth_1962_240827_2124_combined.bpc
➜ Downloading kernel...
    Saving to /Users/thomasruch/.spacerocks/spice/earth_1962_240827_2124_combined.bpc
    Download complete
Loading kernel: /Users/thomasruch/.spacerocks/spice/earth_1962_240827_2124_combined.bpc


In [1]:
from spacerocks.time import Time
from spacerocks.observing import Observatory
from spacerocks.time import Time
from spacerocks import SpaceRock
from spacerocks.spice import SpiceKernel

In [2]:
kernel = SpiceKernel()
kernel.load("/Users/thomasruch/Gerdes/leap_seconds.tls")
kernel.load("/Users/thomasruch/Gerdes/de440s.bsp")
kernel.load("/Users/thomasruch/.spacerocks/spice/earth_1962_240827_2124_combined.bpc")

Loading kernel: /Users/thomasruch/Gerdes/leap_seconds.tls
Loading kernel: /Users/thomasruch/Gerdes/de440s.bsp
Loading kernel: /Users/thomasruch/.spacerocks/spice/earth_1962_240827_2124_combined.bpc


In [3]:
# Create observatory and time
telescope = Observatory.from_obscode('F51')
epoch = Time.now()

# Create and configure target
target = SpaceRock.from_horizons(
    "Apophis", 
    epoch,
    reference_plane="J2000",
    origin='SSB'
)
target.set_absolute_magnitude(19.7)

# Get observation
observer = telescope.at(epoch)
obs = target.observe(observer)

print(f"RA: {obs.ra}")
print(f"Dec: {obs.dec}")
print(f"Range: {obs.range} AU")
print(f"Magnitude: {obs.mag}")

RA: 5.260928972940125
Dec: -0.3281633047446638
Range: 1.7486690602683705 AU
Magnitude: 20.90439792814383


In [4]:
# Set up observatories
mauna_kea = Observatory.from_obscode('568')
siding_spring = Observatory.from_obscode('413')

In [5]:
# Create time series
step = 100
n_points = 10
start_jd = Time.now().jd()
epochs = [Time((start_jd + i*step), 'UTC', 'JD') for i in range(n_points)]
observations = []

In [7]:
for epoch in epochs:
    # Observe from both sites
    target.analytic_at(epoch)
    obs_mk = target.observe(mauna_kea.at(epoch))
    obs_sso = target.observe(siding_spring.at(epoch))
    observations.append((obs_mk, obs_sso))

AttributeError: 'builtins.SpaceRock' object has no attribute 'analytic_at'

In [8]:
# Create SpaceRock
epoch = Time.now()
arrokoth = SpaceRock.from_horizons("Arrokoth", epoch)

# Calculate orbital elements
print(f"Semi-major axis: {arrokoth.a()} AU")
print(f"Eccentricity: {arrokoth.e()}")

# Set physical properties
arrokoth.set_mass(1e-10)
arrokoth.set_absolute_magnitude(15.0)

Semi-major axis: 44.23095719620317 AU
Eccentricity: 0.037802504828944004


In [13]:
from spacerocks.observing import Observer, Observatory, Observation
# Set up objects
epoch = Time.now()
asteroid = SpaceRock.from_horizons("Ceres", epoch, "J2000")
telescope = Observatory.from_obscode("w84")
observer = telescope.at(epoch)  

# Calculate ephemeris
observation = asteroid.observe(observer)
print(f"RA: {observation.ra} rad")
print(f"Dec: {observation.dec} rad")

RA: 5.616024589149868 rad
Dec: -0.39244776678263826 rad


In [21]:
time = Time(2905840.9, "utc", "jd")

In [22]:
time.to_tdb()

Time: 2905840.9008007217 TDB JD

In [23]:
time

Time: 2905840.9008007217 TDB JD

In [26]:
time_in_utc = time.utc()

In [27]:
time

Time: 2905840.9008007217 TDB JD

In [28]:
time_in_utc

Time: 2905840.9 UTC JD

In [30]:
time.mjd()

505840.4008007217

In [32]:
time.calendar()

'27 Oct 3243'

In [33]:
time.iso()

'3243-10-27T09:35:59.000Z'

In [34]:
time

Time: 2905840.9008007217 TDB JD

In [35]:
time + 1000

Time: 2906840.9008007217 TDB JD

In [38]:
time_new = Time(20424.5, 'tdx', 'mjd')

ValueError: Invalid timescale: 'tdx'. Did you mean 'tdb'?. Needs to be 'utc', 'tdb', 'tt', or 'tai'.

In [37]:
time_new

(20424.5, 'tdx', 'mjd')

In [39]:
from spacerocks import MPCHandler

In [40]:
handler = MPCHandler()

In [47]:
# Create a dataframe of all the Near-Earth Asteroids (NEAs) from the MPC data files
nea_df = handler.fetch_data(
    catalog="mpcorb_extended", # https://minorplanetcenter.net/data details available catalogs
    orbit_type= "Hilda",  # None means get all types, and is the default
    output_format="dataframe" # Options: "dataframe", "rocks": Default is "dataframe", "rocks" returns a RockCollection 
)

Using existing json file: /Users/thomasruch/.spacerocks/mpc/mpcorb_extended.json.gz


In [48]:
nea_df

,H,G,Epoch,M,Peri,Node,i,e,a,Principal_desig,orbit_type
0,7.75,0.15,2460600.5,63.64996,39.32434,228.08798,7.82939,0.138727,3.970658,A875 VC,Hilda
1,7.91,0.15,2460600.5,146.33147,271.30376,175.42328,6.17472,0.168317,3.994557,A878 SA,Hilda
2,7.64,0.15,2460600.5,114.77809,169.05178,129.80984,4.63717,0.025093,3.920176,A892 QB,Hilda
3,8.39,0.15,2460600.5,345.48552,67.12248,18.74267,12.58515,0.211862,3.985101,A893 EF,Hilda
4,9.62,0.15,2460600.5,157.46959,175.11255,256.17038,2.09258,0.217457,4.014617,A902 YE,Hilda
...,...,...,...,...,...,...,...,...,...,...,...
6357,17.40,0.15,2458340.5,330.77136,161.57387,206.89741,3.32396,0.189466,3.897767,2018 PU25,Hilda
6358,18.50,0.15,2458340.5,338.79285,219.38379,146.60116,6.88106,0.231140,3.886307,2018 PQ30,Hilda
6359,17.10,0.15,2458760.5,350.95844,251.46347,148.73943,1.41872,0.174703,3.949616,2019 SN122,Hilda
6360,16.90,0.15,2458780.5,339.24413,275.67108,142.23657,3.78468,0.155806,3.924514,2019 US68,Hilda


In [51]:
from spacerocks.orbfit import gauss
from spacerocks.observing import Observer
from spacerocks.time import Time

In [54]:
# Create or load three observations
time1 = Time.now()
time2 = time1 + 150
time3 = time2 + 150

observer1 = Observatory.from_obscode('w84').at(time1)
observer2 = Observatory.from_obscode('w84').at(time2)
observer3 = Observatory.from_obscode('w84').at(time3)

obs1 = Observation.from_astrometry(epoch, 3.45, 0.24, observer1)
obs2 = Observation.from_astrometry(epoch, , dec, observer2)
obs3 = Observation.from_astrometry(epoch, ra, dec, observer3)


NameError: name 'ra' is not defined